# Lecture 2A: Genome indexing - Substring indexing
___

In this lecture, we will introduce the substring indexing algorithm for the traditional method for indexing the reference genome when we do reads alignment against the  genome.

In order to index a string $T$, we need to extract the sequences (usually substrings), along with pointers back to where they occurred.

## 1. `map` data structure: key-value pairs

We usually organize the substrings as well as the pointers into a **map** data structure. Various map structures are available, all involving some mix of sorting/grouping (排序、组合).

### 1.1 Sorted list
Let's make a class that implements an inverted (substring) index where the map data sructure is 
> a sorted list of (substring, offset) pairs, a data structure appropriate for binary search.

In [3]:
import sys
import bisect # implementing binary search

class IndexSorted(object):
    
    def __init__(self, t, ln=2):
        """ Create index, extracting substrings of length 'ln' """
        self.t = t
        self.ln = ln
        self.index = []
        for i in xrange(0, len(t)-ln+1):
            self.index.append((t[i:i+ln], i)) # extract <substr, offset> pair and add into list
        self.index.sort() # sort pairs
    
    def query(self, p):
        """ Return candidate alignments for p """
        st = bisect.bisect_left(self.index, (p[:self.ln], -1)) # find first entries for matching
        en = bisect.bisect_right(self.index, (p[:self.ln], sys.maxint)) # find last entries for matching
        hits = self.index[st:en] # this range of elements corresponds to the hits
        return [ h[1] for h in hits ] # return just the list of offsets

Here is an example:

In [5]:
index = IndexSorted('CGTGCCTACTTACTTACAT', 5)
print index.query('CTTACTTA') # list of 0-based index
print index.query('TTTTTTTT') # list of 0-based index
print index.index

[8, 12]
[]
[('ACTTA', 7), ('ACTTA', 11), ('CCTAC', 4), ('CGTGC', 0), ('CTACT', 5), ('CTTAC', 8), ('CTTAC', 12), ('GCCTA', 3), ('GTGCC', 1), ('TACAT', 14), ('TACTT', 6), ('TACTT', 10), ('TGCCT', 2), ('TTACA', 13), ('TTACT', 9)]


### Query of $P$ against $T$
1. Create the sorted index for the target sequence, $T$. That is, sorted table of `ln`-long substrings, along with their corresponding offsets.
2. Extract `ln`-long prefix for $P$
3. Look up the prefix in the index
4. Investigate all hits

In [11]:
def queryIndex(p, t, ln=2):
    '''look for occurrences of p in t'''
    index = IndexSorted(t, ln)  ## create for t the index of lenth ln
    occurrences = []
    for i in index.query(p):
        if t[i+ln:i+len(p)] == p[ln:]:
            occurrences.append(i)
    return(occurrences)

In [12]:
T = 'CGTGCCTACTTACTTACAT'
P = 'CTTACTTA'
print queryIndex(P, T)

[8]


### <font color="red">$\S$ Exercise 1</font>

For human genome, how much memory would it take for building index of length-4 substrings?  Do you think it can be done on your PC?

###  1.2 Variants of sorted list of (substring, offset)

1. Extract substrings from reference, with a given `length=ln` and `interval=x`, then build the sorted index.
2. Extract first `x` substring for $P$, and  do lookup for each.

In [29]:
import sys
import bisect # implementing binary search

class IndexSorted2(object):
    
    def __init__(self, t, ln=2, interval=2):
        """ Create index, extracting substrings of length 'ln' """
        self.t = t
        self.ln = ln
        self.interval = interval
        self.index = []
        for i in xrange(0, len(t)-ln+1, interval):
            self.index.append((t[i:i+ln], i)) # extract <substr, offset> pair and add into list
        self.index.sort() # sort pairs
    
    def query(self, p):
        """ Return candidate alignments for p """
        st = bisect.bisect_left(self.index, (p[:self.ln], -1)) # find first entries for matching
        en = bisect.bisect_right(self.index, (p[:self.ln], sys.maxint)) # find last entries for matching
        hits = self.index[st:en] # this range of elements corresponds to the hits
        return [ h[1] for h in hits ] # return just the list of offsets
    
def queryIndex2(p, t, ln=2, interval=2):
    '''look for occurrences of p in t with the help of substring index with interval of  length ln'''
    index = IndexSorted2(t, ln, interval)
    occurrences = []
    for k in xrange(0, interval):
        for i in index.query(p[k:]):
            if t[i-k:i] == p[:k] and t[i+ln:i+len(p)-k] == p[k+ln:]:
                occurrences.append(i-k)
    return sorted(occurrences)

In [33]:
T = 'CGTGCCTACTTACTTACAT'
P = 'CTTACTTA'
print queryIndex2(P, T, 4, 4)

[8]


### 1.3 Dictionary-like structure - hash table

Now let's make a similar class but using a Python dictionary instead of a sorted list.  A Python dictionary is basically a hash table.

In [5]:
class IndexHash(object):
    
    def __init__(self, t, ln, ival):
        """ Create index, extracting substrings of length 'ln' """
        self.t = t
        self.ln = ln
        self.ival = ival
        self.index = {}
        for i in xrange(0, len(t)-ln+1):
            substr = t[i:i+ln]
            if substr in self.index:
                self.index[substr].append(i) # substring already in dictionary
            else:
                self.index[substr] = [i] # add to dictionary
    
    def query(self, p):
        """ Return candidate alignments for p """
        return self.index.get(p[:self.ln], [])

And here are some examples for using hash table:

In [10]:
index2 = IndexHash('CGTGCCTACTTACTTACAT', 5, 4)
print index2.query('CTTACTTA') 
print index2.query('TTTTTTTT') 

### 1.4 Trie

A trie is a tree representing a collection of strings with one node per common prefix. 

A trie is the smallest tree such that:
   - Each key is "spelled out" along some path starting at the root. 
   - Each edge is labeled with a character $c \in \Sigma$. 
   - A node has at most one outgoing edge labeled $c$, for $c \in \Sigma$. 

Trie is a natural way to represent either a set or a map where keys are strings.

We can index $T$ with a trie, like this:

![](images/trie.png)

In [13]:
class TrieMap(object):
    """ Trie implementation of a map.  Associating keys (strings or other
        sequence type) with values.  Values can be any type. """
    
    def __init__(self, kvs):
        self.root = {}
        # For each key (string)/value pair
        for (k, v) in kvs: self.add(k, v)
    
    def add(self, k, v):
        """ Add a key-value pair """
        cur = self.root
        for c in k: # for each character in the string
            if c not in cur:
                cur[c] = {} # if not there, make new edge on character c
            cur = cur[c]
        cur['value'] = v # at the end of the path, add the value
    
    def query(self, k, ln):
        """ Given key, return associated value or None """
        cur = self.root
        for c in k[:ln]:
            if c not in cur:
                return None # key wasn't in the trie
            cur = cur[c]
        # get value, or None if there's no value associated with this node
        return cur.get('value')

In [19]:
kvs =[('ACTTA', 7), ('ACTTA', 11), ('CCTAC', 4), ('CGTGC', 0), ('CTACT', 5), ('CTTAC', [8, 12]), ('GCCTA', 3), ('GTGCC', 1),  ('TACAT', 14), ('TACTT', 6), ('TACTT',10), ('TGCCT', 2), ('TTACA', 13), ('TTACT', 9)]
genome = TrieMap(kvs)
genome.query('CTTACACT', 5)


[8, 12]

There are many other tree structure to store keys or maps with string keys, such as binary or ternary search trees.

#### Reference
Bentley, Jon L., and Robert Sedgewick. **Fast algorithms for sorting and searching strings**. Proceedings of the eighth annual ACM-SIAM symposium on Discrete algorithms. 1997

### Summary

All the above methods are based upon key-value pairs of the **length=ln** subtrings and their offsets:
>  [(key, offset) for key in substr(genome, ln)]

But we use different data structure to store the keys. And these data structures have different properties.

1. Sorted list: $\mathcal{O}(\log m)$ query time, only stores keys (substring of length `ln`) and values (offsets);
2. Hash: $\mathcal{O}(1)$ query time, but need to store keys, values, bucket array, and pointers.
3. Trie:  $\mathcal{O}(n)$ query time, need to store the tree structure.

where $m, n$ are the respective lengths for the corresponding genome and read.

### <font color="red">$\S$ Exercise 2</font>

1. Tell the memory requirement for the above indexing data structures for a human genome.
2. Which indexing method does BLAST, the popular alignment tool, adopt?

In the next lecture, we will introduce the **suffix-based indexing**.